<a href="https://colab.research.google.com/github/cassidywilliams/realtime-digit-predictor/blob/master/training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

Using TensorFlow backend.


In [0]:
batch_size = 256
num_classes = 10
epochs = 12

In [0]:
img_rows, img_cols = 28, 28

(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [0]:
def create_cnn_model(num_classes):
  

    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3),
                    activation='relu',
                    input_shape=input_shape))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

    
    return model

In [0]:
model = create_cnn_model(num_classes)

In [0]:
def fit_eval_cnn_model(x_train, y_train, x_test, y_test, batch_size, epochs):
    model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
    score = model.evaluate(x_test, y_test, verbose=0)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])

In [0]:
fit_eval_cnn_model(x_train, y_train, x_test, y_test, batch_size, epochs)

Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 13s 212us/step - loss: 0.3624 - accuracy: 0.8880 - val_loss: 0.0769 - val_accuracy: 0.9746
Epoch 2/12
60000/60000 [==============================] - 7s 109us/step - loss: 0.1096 - accuracy: 0.9673 - val_loss: 0.0564 - val_accuracy: 0.9828
Epoch 3/12
60000/60000 [==============================] - 7s 109us/step - loss: 0.0817 - accuracy: 0.9757 - val_loss: 0.0470 - val_accuracy: 0.9859
Epoch 4/12
60000/60000 [==============================] - 7s 109us/step - loss: 0.0663 - accuracy: 0.9800 - val_loss: 0.0390 - val_accuracy: 0.9865
Epoch 5/12
60000/60000 [==============================] - 6s 107us/step - loss: 0.0579 - accuracy: 0.9829 - val_loss: 0.0400 - val_accuracy: 0.9875
Epoch 6/12
60000/60000 [==============================] - 6s 107us/step - loss: 0.0498 - accuracy: 0.9854 - val_loss: 0.0308 - val_accuracy: 0.9899
Epoch 7/12
60000/60000 [==============================] - 7s 

In [0]:
model.save('initial_mnist.h5')